In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Usable Code

In [ ]:
!pwd

In [ ]:
files = sorted([f for f in os.listdir('csv') if '.csv' in f])

In [ ]:
files

In [ ]:
def clean_odds(file):
    '''
    cleans csv file and returns dataframe object that contains the over/under lines (target)
    '''
    df = pd.read_csv(file)
    df = df.iloc[:,:13] #some files contained unnamed extra rows
    df.dropna(axis=0,inplace=True) #and others columns
    
    #makes a list of game_id's equivalent to how many games were played that season
    mylist = []
    for i in range(1, int(df.shape[0]/2 + 1)):
        mylist.append(i)
        mylist.append(i)
    df['game_id'] = mylist #maps game_id's to games (spread across 2 rows)
    
    df = df.merge(df, on='game_id', suffixes=(None,'_2')) #gets games to be in one row (creates 4/game)
    df = df[1::4] #gets correct mapping
    df.reset_index(drop=True, inplace=True)
    
    #make dates include year so we can join with another dataset later
    df['Date'] = df['Date'].astype('int')
    df['Date'] = df['Date'].astype('str')
    #df['Date'] = df['Date'].str.strip('.0')
    dates = []
    for date in df['Date']:
        if int(date) > 1000:
            date = '20'+file[:2]+date #GET FILENAMES AND USE HERE
            dates.append(date)
        else:
            date = '20'+file[3:5]+'0'+date #GET FILENAMES AND USE HERE
            dates.append(date)
    df['Date'] = dates
    
    #take out pick-em's and replace with zero for later transformation
    df['Open'].replace(['PK','pk'],'0',inplace=True)
    df['Close'].replace(['PK','pk'],'0',inplace=True)
    df['Open_2'].replace(['PK','pk'],'0',inplace=True)
    df['Close_2'].replace(['PK','pk'],'0',inplace=True)
    df.replace('197.5u10','197.5',inplace=True) #one unique occurrence
    #map strings as floats for comparison
    df['Open'] = df['Open'].astype('float')
    df['Open_2'] = df['Open_2'].astype('float')
    df['Close'] = df['Close'].astype('float')
    df['Close_2'] = df['Close_2'].astype('float')
    #get correct over/under line at opening and closing of sportsbook
    df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])
    df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])
    
    #rename columns to correct home/visitor 
    df['Visitor'] = df['Team']
    df['Home'] = df['Team_2']
    
    #add season column
    df['Season'] = file[:2] + file[3:5]
    
    #drop unneeded info
    df = df[['Date','Home','Visitor','O/U_open','O/U_close','Season']]
    return df

Use this clean_odds function to generate our dataset to combine with our scraped game data.

In [ ]:
cd csv

In [ ]:
lines = pd.concat([clean_odds(f) for f in files])

In [ ]:
lines

In [ ]:
lines.reset_index(drop=True,inplace=True)
lines

Let's pickle this initial processing of the csv's and subsequent DataFrame. We can always comeback to this if we need it.

In [ ]:
with open('lines.pickle', 'wb') as to_write:
    pickle.dump(lines, to_write)

In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
with open('game_df.pickle','rb') as read_file:
    game_df = pickle.load(read_file)
    
game_df

In [ ]:
team_abbrev = game_df['home'].unique()
team_abbrev

In [ ]:
len(team_abbrev)

In [ ]:
lines['Home'].unique()

In [ ]:
team_dict = {
    'SAS': 'SanAntonio',
    'GSW': 'GoldenState',
    'LAL': 'LALakers',
    'TOR': 'Toronto',
    'IND': 'Indiana',
    'ORL': 'Orlando',
    'NJN': 'NewJersey',
    'CLE': 'Cleveland',
    'MEM': 'Memphis',
    'NOH': 'NewOrleans',
    'NOP': 'NewOrleans',
    'DEN': 'Denver',
    'MIA': 'Miami',
    'UTA': 'Utah',
    'SEA': 'Seattle',
    'CHA': 'Charlotte',
    'CHO': 'Charlotte',
    'ATL': 'Atlanta',
    'BOS': 'Boston',
    'MIN': 'Minnesota',
    'CHI': 'Chicago',
    'PHO': 'Phoenix',
    'LAC': 'LAClippers',
    'PHI': 'Philadelphia',
    'WAS': 'Washington',
    'MIL': 'Milwaukee',
    'HOU': 'Houston',
    'DAL': 'Dallas',
    'NYK': 'NewYork',
    'DET': 'Detroit',
    'SAC': 'Sacramento',
    'POR': 'Portland',
    'OKC': 'OklahomaCity',
    'BRK': 'Brooklyn'
}

In [ ]:
game_df['home'].map(team_dict)

In [ ]:
game_df['home_team'] = game_df['home'].map(team_dict)

In [ ]:
game_df

In [ ]:
game_df['game_date'] = game_df['game_id'].apply(lambda x: x[:8])

In [ ]:
game_df

In [ ]:
game_df['id'] = game_df['game_date'] + game_df['home_team']

In [ ]:
game_df

In [ ]:
lines['id'] = lines['Date'] + lines['Home']

In [ ]:
lines

In [ ]:
lines.info()

In [ ]:
game_df.info()

Now both databases have a column to merge on. The 'id' column.

In [ ]:
df = pd.merge(lines,game_df,on='id')

In [ ]:
df

In [ ]:
df.shape

Drop columns that share the same information.

In [ ]:
df.drop(columns=['Home','Visitor','game_id','home_team','game_date'],inplace=True)

In [ ]:
df

Getting the difference between the line and the outcome of a particular game in order to classify our game as an "Over" or "Under".

In [ ]:
df['ou1'] = df['total'] - df['O/U_open']
df['ou2'] = df['total'] - df['O/U_close']

Function to label our target based off information we have in our DataFrame, and then applying that function to our newly created columns that currently classify our games based on a positive or negative value. This simply puts a more general categorical label over that more granular scalar indicator.

In [ ]:
def over_under(ou):
    if ou > 0:
        return 1
    elif ou == 0:
        return 2
    else:
        return 0

In [ ]:
df['Over/Under_open'] = df['ou1'].apply(over_under)
df['Over/Under_close'] = df['ou2'].apply(over_under)

CRITICAL ASSUMPTION: Making the decision to drop "pushes". A "push" is when a betting line is hit exactly, and no money exchanges hands. In the context of this project, a total could be at 212 points, and if the game finishes with exactly 212 points, then every bettor gets their money back, on both sides (over/under 212), and the sportsbook doesn't collect anything.

In [ ]:
df = df[df['Over/Under_open'] != 2]
df = df[df['Over/Under_close'] != 2]

About 400 instances of a "push", or about 2.7% of our original dataset. This number can be remembered for sampling or simulation purposes later.

Convert date to pd.datetime object. Makes plotting the time series compatible with matplotlib, and building season dictionary later.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

### Stat building (feature engineering)

We want to ultimately end up with a model that takes both teams' recent performances and can make a prediction on the total, so we can make a new column to hold home/visitor every other row, and then rename the _h and _v columns for self, opponent.

In [ ]:
d = df.reindex(df.index.repeat(2)).reset_index(drop=True)

Now that we have every game repeated, let's create a column to indicate which team these self/opponent stats represent. We can do this by creating an empty column and selectively copy from the 'home' and 'away' columns that repeat across the 2 rows. Then we'll do that same for the opponent.

In [ ]:
d['team'] = np.nan
#home team will be all even indexes of this dataset
d['team'][::2] = d['home'][::2]
#away team will be all odd indexes of this dataset
d['team'][1::2] = d['away'][1::2]

In [ ]:
d['opp'] = np.nan
d['opp'][::2] = d['away'][::2]
d['opp'][1::2] = d['home'][1::2]

Let's keep track of home/away, can't hurt. We'll use the same process as we just used. Note: this can also be used as a sanity check to make sure that our splitting worked correctly. If it did, we should see 'H', 'V' every other row.

In [ ]:
d['home/away'] = np.nan
#home team will be all even indexes of this dataset
d['home/away'][::2] = 'H'
#away team will be all odd indexes of this dataset
d['home/away'][1::2] = 'V'

Now we have to map the _v and _h stats appropriate to new columns for self or opponent stats.

In [ ]:
stats = ['pace','eFg','tov','orb','ft_fga','ortg','fg','fga','fg_per','threes','threes_att','threes_per',
         'ft','fta','ft_per','drb','trb','ast','stl','blk','to','fouls','ts_per','threes_ar','ft_ar',
         'drb_per','trb_per','ast_per','stl_per','blk_per','user_per','drtg']
for stat in stats:
    d['{}'.format(stat)] = ""
    d['{}'.format(stat)][::2] = d['{}_h'.format(stat)][::2]
    d['{}'.format(stat)][1::2] = d['{}_v'.format(stat)][1::2]
    d['{}_opp'.format(stat)] = ""
    d['{}_opp'.format(stat)][::2] = d['{}_v'.format(stat)][::2]
    d['{}_opp'.format(stat)][1::2] = d['{}_h'.format(stat)][1::2]

Then we can drop all of our columns with '_v' & '_h' since they contain extra, and now, redundant information.

In [ ]:
mylist = []
for stat in stats:
    mylist.append('{}_v'.format(stat))
    mylist.append('{}_h'.format(stat))
#print(mylist)
d = d.drop(columns=mylist)

Now our DataFrame (d) has stats for each team and their opponent for every game. Now let's get some rolling averages. First we'll create an empty column that we'll populate with a rolling count for every time that team has appeared. This will get us the number of games each team has played up INCLUDING that game for each season. We can use that info to incorporate some rolling averages.

In [ ]:
d['team_season'] = d['team'] + d['Season']

In [ ]:
d['game_num'] = d.groupby('team_season').cumcount()+1

Now we are ready to compute some rolling average's in a certain season. Remember, we want to get a snapshot of how both team's have perfromed recently (last 5 games), and get a classification from this info. So let's go back to our method of creating empty columns we can assign data to.

In [ ]:
for stat in d.iloc[:,15:79]:
    d['{}_rolling'.format(stat)] = np.nan

Cast our stat columns as floats to perform some operations on them.

In [ ]:
for col in d.iloc[:,15:79].columns:
    d[col] = d[col].astype('float')

This may not be the most pythonic way to populate these new columns, but it works, and doesn't take long. At a high level, we are going team by team, then season by season, for that team, and applying 5 game rolling averages to their stats and opponent stats.

In [ ]:
for team in d['team'].unique():
    mask = d['team'] == team
    d4 = d[mask]
    for season in d4['Season'].unique():
        mask = d4['Season'] == '{}'.format(season)
        d5 = d4[mask]
        for stat in list(d5.iloc[:,15:79].columns):
            d5['{}_rolling'.format(stat)] = d5.rolling(window=5)['{}'.format(stat)].mean().shift(1)
        d.update(d5)

IMPORTANT NOTE: this DataFrame contains rows with NaN's. Every game that is in the first 5 of the season has no rolling averages, this can be problematic when we take this DataFrame and try to feed it into a sk-learn ML model. We can address this now, by simply dropping these games(~5% of our dataset), or apply the previous season's median rolling average. This is where some decisions need to be made, and I would advise just dropping the games for 2 reasons. One, taking out 5 games won't change the application our model will have as there are 67-75 games left in the year we can apply this model to once in production. Two, a median of the rolling averages, or any other kind of congregation statistic being applied over different seasons is mostly likely not a sound choice in light of team personnel turnover, from retirings, trades, the draft, and free agent moves; not to mention new coaching staffs. This could be applicable to a few teams (ones that experience little of this roster turnover), but not enough to apply that thinking across the entire dataset.

In [ ]:
d

We won't cut off these rows in this notebook, but can easily do so in our modeling notebook by using the following mask and declaring this filtered d as our new d.

In [ ]:
d[d['game_num']>5]

The final thing we have to do before we can begin classifying our games is make each row one game. Our features will include rolling averages for each of our team's and opponent's stats. From here we can get a baseline sense if our logic, of recent past performances, is good basis for classification.

In [ ]:
d

Now that our data is fully processed, we are ready to process this in a classification algorithm

In [ ]:
with open('d_rolling.pickle', 'wb') as to_write:
    pickle.dump(d, to_write)

## Scratch Work

In [ ]:
df = pd.read_csv(files[2])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.iloc[:,:13]

In [ ]:
df.info()

In [ ]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

In [ ]:
df.shape[0]

In [ ]:
mylist = []
for i in range(1, int(df.shape[0]/2 + 1)):
    mylist.append(i)
    mylist.append(i)
print(mylist[0],mylist[-1])

In [ ]:
len(range(1,int(df.shape[0]/2 + 1)))

In [ ]:
df['game_id'] = mylist

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df.info()

In [ ]:
df = df.merge(df, on='game_id', suffixes=(None,'_2'))

In [ ]:
df

In [ ]:
df = df[1::4]
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
#sanity check that every game only appears once
df['game_id']

In [ ]:
df['Date'] = df['Date'].astype('int')
df['Date']

In [ ]:
df['Date'] = df['Date'].astype('str')
df['Date']

In [ ]:
df['Date'] = df['Date'].str.strip('.0')

In [ ]:
df['Date'].head

In [ ]:
dates = []
for date in df['Date']:
    if int(date) > 1000:
        date = '20'+files[2][:2]+date
        dates.append(date)
    else:
        date = '20'+files[2][3:5]+'0'+date
        dates.append(date)
df['Date'] = dates

In [ ]:
df['Date']

In [ ]:
df

In [ ]:
df['Visitor'] = df['Team']
df['Home'] = df['Team_2']
df

In [ ]:
df['Open'].replace('pk','0',inplace=True)
df['Close'].replace('pk','0',inplace=True)
df['Open_2'].replace('pk','0',inplace=True)
df['Close_2'].replace('pk','0',inplace=True)
df.replace('197.5u10','197.5',inplace=True)

In [ ]:
mask = df['Open'].values == 'pk'
df[mask]

In [ ]:
df['Open'] = df['Open'].astype('float')
df['Open_2'] = df['Open_2'].astype('float')
df['Close'] = df['Close'].astype('float')
df['Close_2'] = df['Close_2'].astype('float')

In [ ]:
df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])
df

In [ ]:
df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])
df

Sanity check that opening and closing lines operations worked successfully.
So long as numbers are around 200, we know we are ok.

In [ ]:
df['O/U_open'].min()

In [ ]:
df['O/U_close'].min()

In [ ]:
df = df[['Date','Home','Visitor','O/U_open','O/U_close']]
df

### Processing our lines df

First, and most importantly, we must create our target labels. This will be done by comparing the total the the opening and closing lines, and mapping that result to one of three categories: Over, Under, or Push. This will represent the winning result of that game.

In [ ]:
df

In [ ]:
df['ou1'] = df['total'] - df['O/U_open']
df['ou2'] = df['total'] - df['O/U_close']

In [ ]:
df

In [ ]:
def over_under(ou):
    if ou > 0:
        return 1
    elif ou == 0:
        return 2
    else:
        return 0

In [ ]:
df['Over/Under_open'] = df['ou1'].apply(over_under)
df['Over/Under_close'] = df['ou2'].apply(over_under)

In [ ]:
df

Making the decision to drop pushes as no money exchanges hands in this scenerio.

In [ ]:
df = df[df['Over/Under_open'] != 2]
df = df[df['Over/Under_close'] != 2]

In [ ]:
df

About 400 instances of a push, or about 2.7% of our original dataset. This number can be remembered for sampling or simulation purposes later.

### Stat Building

We have the same set of stats for both teams in any one game, so we can build offense/defense for both teams.

In [ ]:
'''map stats accordingly:
1. Get 2 "sets" of stats per game:

visitor: offense - _v stat avgs heading into the game; defense - _h stat avgs heading into the game
home: offese - _h stat avgs heading into the game; defense - _v stat avgs heading into the game

2. Map visitor/home stats to respective teams

3. Build dictionary of team's seasons to be further processed.

{1415: {GSW: {..game_35:{offense/defense stats},game_36:{..}

4. Process dictionary to have more stats/potential model features:

{GSW: {..game_35:{offense/defense stats averaged through 34 games},game_36:{..}

5. ?

'''

In [ ]:
df.info()

Convert date to pd.datetime object. May help with building season dictionary later.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [ ]:
df['Date']

In [ ]:
d = df.reindex(df.index.repeat(2)).reset_index(drop=True)

Now that we have every game twice, we can make a new column to hold home/visitor every other row, and then rename the _h and _v columns for self, opponent.

In [ ]:
d

Let's create the column we want to place our values, then selectively copy.

In [ ]:
d['team'] = np.nan

In [ ]:
#home team will be all even indexes of this dataset
d['team'][::2] = d['home'][::2]

In [ ]:
#away team will be all odd indexes of this dataset
d['team'][1::2] = d['away'][1::2]

In [ ]:
d['opp'] = np.nan

In [ ]:
d['opp'][::2] = d['away'][::2]
d['opp'][1::2] = d['home'][1::2]

In [ ]:
d[['team','opp']]

Let's keep track of home/away, can't hurt. We'll use the same process as we just used.

In [ ]:
d['home/away'] = np.nan
#home team will be all even indexes of this dataset
d['home/away'][::2] = 'H'
#away team will be all odd indexes of this dataset
d['home/away'][1::2] = 'V'

In [ ]:
d.info()

Now we have to map the _v and _h stats appropriate to new columns for self or opponent stats.

In [ ]:
stats = ['pace','eFg','tov','orb','ft_fga','ortg','fg','fga','fg_per','threes','threes_att','threes_per',
         'ft','fta','ft_per','drb','trb','ast','stl','blk','to','fouls','ts_per','threes_ar','ft_ar',
         'drb_per','trb_per','ast_per','stl_per','blk_per','user_per','drtg']
for stat in stats:
    d['{}'.format(stat)] = ""
    d['{}'.format(stat)][::2] = d['{}_h'.format(stat)][::2]
    d['{}'.format(stat)][1::2] = d['{}_v'.format(stat)][1::2]
    d['{}_opp'.format(stat)] = ""
    d['{}_opp'.format(stat)][::2] = d['{}_v'.format(stat)][::2]
    d['{}_opp'.format(stat)][1::2] = d['{}_h'.format(stat)][1::2]

In [ ]:
mylist = []
for stat in stats:
    mylist.append('{}_v'.format(stat))
    mylist.append('{}_h'.format(stat))
#print(mylist)
d = d.drop(columns=mylist)

In [ ]:
d.info()

Now our DataFrame (d) has stats for each team and their opponent for every game. Now let's get some rolling averages.

First we'll create an empty column that we'll populate with a rolling count for every time that team has appeared. This will get us the number of games each team has played up INCLUDING that game for each season. We can use that info to incorporate some rolling averages.

In [ ]:
d['team_season'] = d['team'] + d['Season']

In [ ]:
d

In [ ]:
d['game_num'] = d.groupby('team_season').cumcount()+1
d

In [ ]:
d.info()

Cast our stat columns as floats to perform some operations on them.

In [ ]:
for col in d.iloc[:,15:79].columns:
    d[col] = d[col].astype('float')

In [ ]:
d.info()

In [ ]:
d['team_season_game_num'] = d['team_season'] + d['game_num'].astype('str')

In [ ]:
d['team_season_game_num']

In [ ]:
for stat in d.iloc[:,15:79]:
    d['{}_rolling'.format(stat)] = np.nan

In [ ]:
d.info()

In [ ]:
for team in d['team'].unique():
    mask = d['team'] == team
    d4 = d[mask]
    for season in d4['Season'].unique():
        mask = d4['Season'] == '{}'.format(season)
        d5 = d4[mask]
        for stat in list(d5.iloc[:,15:79].columns):
            d5['{}_rolling'.format(stat)] = d5.rolling(window=5)['{}'.format(stat)].mean().shift(1)
        d.update(d5)

In [ ]:
d[d['game_num']>5]

For each team, get a single season, calculate rolling averages, then insert into our DataFrame (d). Necessary since every season has potentially different number of games

In [ ]:
for season in d4['Season'].unique():
    mask = d4['Season'] == '{}'.format(season)
    d5 = d4[mask]
    d5
    for stat in list(d5.iloc[:,15:79].columns):
        d5['{}_rolling'.format(stat)] = d5.rolling(window=5)['{}'.format(stat)].mean().shift(1)
    #print(len(d4[d4['Season'] == '{}'.format(season)]))

In [ ]:
d5.columns[82:146]

In [ ]:
d5.index.values

In [ ]:
d5

In [ ]:
d.update(d5)

In [ ]:
d.iloc[28947:28970,:]

In [ ]:
stat_list = list(d3.iloc[:,5:].columns)

In [ ]:
for stat in stat_list:
    d3['{}_rolling'.format(stat)] = d3.rolling(window=5)['{}'.format(stat)].mean().shift(1)

In [ ]:
d3

## Scratch Work Graveyard

Processing to merge datasets graveyard

In [ ]:
#tried self-merging on game_id, but this is only applicable if in different columns
#df1 = df1.merge(
#            right=df1[opp_pull_cols],
#            left_on=["game_id", "team"],
#            right_on=["game_id", "opp"],
#            suffixes=[None, "_opp"],
#        )

In [ ]:
#tried to use drop_duplicates method but can only keep first or last
#df.drop_duplicates(subset=['game_id'], keep='second')

Rolling average graveyard

In [ ]:
#found online, talks about rolling average based on multiple columns, never tried
#df.loc[:, 'value_sma_10'] = df.groupby(by='object')[['object', 'period']].rolling(window=10, min_periods=1, on='period').mean().reset_index(level='object')['value']

In [ ]:
#found online, got it to work, but not quite applicable to this situation
#span = 5
#sma = d2.rolling(window=span, min_periods=span).mean()[:span]
#rest = d2[span:]
#pd.concat([sma, rest]).ewm(span=span, adjust=False).mean()

In [ ]:
#didn't work - ValueError: cannot reindex from a duplicate axis
#d2['eFg_rolling'] = d2.groupby(['team_season','game_num'])['eFg'].rolling(10).mean().droplevel(level=[0])

In [ ]:
#df1 = d.copy()
#df1

In [ ]:
#df1 = d.groupby(['team_season','game_num']).rolling(5)['eFg'].mean().reset_index(drop=True)

In [ ]:
#df1['pace_rolling'] = d.groupby(['team_season','game_num'])[5:,'pace'].transform(lambda x: x.rolling(10, 10).mean())